In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
print(torch.__version__)
print(pd.__version__)
print(np.__version__)

2.9.1+cpu
2.3.3
2.3.3


In [6]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)


Usando dispositivo: cpu


In [8]:
import urllib.request
import zipfile
import os

# URL del dataset
url = "https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip"

# Nombre del archivo zip local
zip_path = "data.zip"

# Descargar
print("Descargando dataset...")
urllib.request.urlretrieve(url, zip_path)
print("Descarga completa!")

# Descomprimir
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("data")
print("Descompresión completa!")

# Verificar estructura
print("Contenido de 'data':", os.listdir("data"))


Descargando dataset...
Descarga completa!
Descompresión completa!
Contenido de 'data': ['data']


In [9]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Transformaciones indicadas en el homework
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Dataset
train_dataset = datasets.ImageFolder("data/data/train", transform=train_transforms)
test_dataset  = datasets.ImageFolder("data/data/test", transform=train_transforms)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=20, shuffle=False)

# Verificar
print("Clases:", train_dataset.classes)
print("Número de imágenes en train:", len(train_dataset))
print("Número de imágenes en test:", len(test_dataset))


Clases: ['curly', 'straight']
Número de imágenes en train: 800
Número de imágenes en test: 201


In [10]:
import torch.nn as nn
import torch

class HairCNN(nn.Module):
    def __init__(self):
        super(HairCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)  # 3 canales -> 32 filtros
        self.pool = nn.MaxPool2d(2,2)
        self.relu = nn.ReLU()
        # Flatten se hace en forward
        self.fc1 = nn.Linear(32*99*99, 64)  # después del pool, tamaño 99x99
        self.fc2 = nn.Linear(64, 1)         # salida 1 neurona
    
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instanciar modelo
model = HairCNN().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)


In [11]:
total_params = sum(p.numel() for p in model.parameters())
print("Total parameters:", total_params)


Total parameters: 20073473


In [13]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}")



Epoch 1/10, Loss: 0.6462, Acc: 0.6362
Epoch 2/10, Loss: 0.5846, Acc: 0.6875
Epoch 3/10, Loss: 0.5270, Acc: 0.7275
Epoch 4/10, Loss: 0.5153, Acc: 0.7475
Epoch 5/10, Loss: 0.4194, Acc: 0.8087
Epoch 6/10, Loss: 0.3849, Acc: 0.8313
Epoch 7/10, Loss: 0.3245, Acc: 0.8525
Epoch 8/10, Loss: 0.2537, Acc: 0.8850
Epoch 9/10, Loss: 0.2775, Acc: 0.8825
Epoch 10/10, Loss: 0.1521, Acc: 0.9475


In [14]:
import numpy as np

median_acc = np.median(history['acc'])
print("Mediana de training accuracy:", median_acc)


Mediana de training accuracy: 0.8200000000000001
